In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

/home/legolas/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
mnist


Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fd2ca6f0198>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fd2ca6f00f0>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fd2ca6f0278>)

In [ ]:
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = session.run(prediction, feed_dict={xs:v_xs, keep_prob:1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = session.run(accuracy, feed_dict={xs:v_xs, ys:v_ys, keep_prob:1})
    return result


In [ ]:
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape))

def bias_variable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))


In [ ]:
# stride [1, x_movement, y_movement, 1]

def conv2d(x,W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


In [ ]:
xs = tf.placeholder(tf.float32, [None, 784])
ys = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs, [-1, 28, 28, 1])
print(x_image.shape)


In [ ]:
#conv_layer1

W_conv1 = weight_variable([5,5,1,32])  # patch 5x5, in size, out size
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) #28x28x32
h_pool1 = max_pool_2x2(h_conv1)   #14x14x32

In [ ]:
#conv_layer2

W_conv2 = weight_variable([5,5,32,64])  # patch 5x5, in size, out size
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)  #14x14x64
h_pool2 = max_pool_2x2(h_conv2)                  #7x7x64

In [ ]:
# fc1 layer

W_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_variable([1024])
# [n_samples, 7, 7, 64] ->> [n_samples, 7*7*64]
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [ ]:
# fc2 layer

W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [ ]:
# the error between prediction and real data

cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [ ]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [ ]:
for i in range(10):
    batch_xs, batch_ys = mnist.train.next_batch(10)
    session.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5})
    if i % 2 == 0:
        print(compute_accuracy(
            mnist.test.images[:10], mnist.test.labels[:10]))
